# COVID-19 for correlation time series in Castilla la Mancha #
Notebook realizado para la asignatura Desarrollo de Sistemas Inteligentes, del Máster en Ingeniería Informática de la Universidad de Castilla la Mancha.
En dicho Notebook se aplicarán técnicas de correlación de series temporales empleando Pearson y DTW (Dynamic Time Warping, enfocado a la investigación del COVID-19, causada por el virus SARS-CoV-2, sobre la población de Castilla la Mancha, y más específicamente sobre los hospitales de la región.

Al aplicar técnicas de correlación, podremos detectar posibles brotes dentro de una provincia, o si aumenta/disminuye el índice de mortalidad si comparamos las variables de casos detectados y fallecidos, por ejemplo.

Realizado por:
*   Miguel de la Cal Bravo
*   Félix Ángel Martínez Muela

## Librerías y dependencias necesarias ##

En primer lugar, importaremos todas aquellas librerías y dependencias necesarias del proyecto.
Entre ellas podemos destacar algunas muy conocidas como son Pandas, Numpy, Plot/Plotly, etc.

In [0]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

!pip install pmdarima
!pip install statsmodels --upgrade

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
df = pd.read_csv('fallecidos.csv',encoding = "ISO-8859-1", index_col=0, header=None).T  # Lectura del fichero de entrada y transpuesta del mismo
df = df.rename(columns={'Fecha': 'Date'}) # Ajuste del nombre de las columnas
df['Date']= pd.to_datetime(df['Date']) # Conversión a tipo fecha
df.set_index('Date', inplace=True) # La columna Fechas como índice
df=df.apply(pd.to_numeric) # Hacer que los tipos de datos sean numéricos
df = df.sort_values('Date') # Ordenar en función de la fecha
df=df.interpolate('zero', fill_value=0, limit_direction='backward') # A 0 los primeros valores antes de un número
df=df.interpolate(method='linear', axis=0).ffill().bfill() # Valores intermedios los interpolas
df

## Pearson correlation ##
A continuación, definimos la función para calcular el coeficiente de Pearson.

In [0]:
!pip install dtw
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
sns.set_context('talk',font_scale=.8)

def pearson(df, col1,col2,do_print):
  overall_pearson_r = df.corr().iloc[df.columns.get_loc(col1),df.columns.get_loc(col2)]
  # Si quiesiéramos imprimir las gráficas, pondríamos la variable do_print a true
  if do_print==True:
    print(f"Pandas computed Pearson r: {overall_pearson_r}")
    f,ax=plt.subplots(figsize=(14,3))
    df.rolling(window=30,center=True).median().plot(ax=ax)
    ax.set(xlabel='Frame',ylabel='Smiling evidence',title=f"Overall Pearson r = {np.round(overall_pearson_r,2)}");
  
    # Set window size to compute moving window synchrony.
    r_window_size = 120
    # Compute rolling window synchrony
    rolling_r = df[col1].rolling(window=r_window_size, center=True).corr(df[col2])
    f,ax=plt.subplots(2,1,figsize=(14,6),sharex=True)
    df.rolling(window=30,center=True).median().plot(ax=ax[0])
    ax[0].set(xlabel='Fecha',ylabel='Hospitalizados')
    rolling_r.plot(ax=ax[1])
    ax[1].set(xlabel='Fecha',ylabel='Hospitalizados')
    plt.suptitle("Smiling data and rolling window correlation")
  return overall_pearson_r

## Dynamic Time Warping ##
A continuación, definimos la función correspondiente al cálculo de DTW, más conocido como Dynamic Time Warping

In [0]:
from dtw import dtw,accelerated_dtw
def dynamicTime(df, col1, col2, do_print):
  d1 = df[col1].values
  d2 = df[col2].values
  d, cost_matrix, acc_cost_matrix, path = accelerated_dtw(d1,d2, dist='euclidean')  
  if do_print==True:
    plt.imshow(acc_cost_matrix.T, origin='lower', cmap='gray', interpolation='nearest')
    plt.plot(path[0], path[1], 'w')
    plt.xlabel('Subject1')
    plt.ylabel('Subject2')
    plt.title('DTW Minimum Path')
    plt.show()
  return d

**Pearson Coefficient**


In [0]:
pearson_dict = dict(pearson_value=0)
for i in df.columns:
  for j in df.columns:
    if(i!=j):
      pearson_val = pearson(df,i,j,False)
      pearson_val= np.abs(pearson_val)
      if(pearson_dict['pearson_value'] < pearson_val):
        pearson_dict= dict(pearson_value=pearson_val, i_value=i, j_value=j)
      
print(pearson_dict)

**Dynamic Time Warping**

In [0]:
dtw_dict=dict(dtw_value=0)
for i in df.columns:
  for j in df.columns:
    if(i!=j):
      dtw_val = dynamicTime(df,i,j,False)
      if(dtw_dict['dtw_value'] < dtw_val):
        dtw_dict= dict(dtw_value=dtw_val, i_value=i, j_value=j)

print(dtw_dict)